In [1]:
pip install folium

In [2]:
pip install geopandas

In [3]:
import folium
import json
import csv
import requests
import pandas as pd
import math
import shapely
from shapely.geometry import shape, GeometryCollection, Point, Polygon
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np

In [4]:
vAPIKEY=''
with open('mapboxkeys.csv', mode='r', ) as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        for row in csv_reader:
            vAPIKEY=row["APIKEY"]

In [5]:
def request_isochrone(lati,long,time, apikey):
    URL = "https://api.mapbox.com/isochrone/v1/mapbox/walking/"+str(long)+","+str(lati)+"?contours_meters="+str(time)+"&contours_colors=6706ce&access_token="+apikey
    r = requests.get(url = URL)
    data = r.json()
    df = pd.json_normalize(data['features'])
    return Polygon(df["geometry.coordinates"][0])

In [6]:
vlc_laundry = pd.read_csv('lavanderias.csv', sep=',', decimal='.')
poi_tratados = pd.read_csv('poi_tratado.csv', sep=',', decimal='.')

In [7]:
dataset_ine_valencia = pd.read_csv('dataset_ine_valencia.csv', sep=',', decimal='.')

In [8]:
poi_tratados.head()

,Unnamed: 0,name,rating,user_ratings_total,latitude,longitude,type,coddistsec,subtypes
0,0,Parvulario El Belén,0.0,0,39.462611,-0.382058,education,336,school
1,1,Centro Privado de Educación Infantil Bambini,3.9,7,39.470982,-0.383840,education,301,school
2,2,Centro Educación Infantil Bambini 2,4.9,10,39.460225,-0.368574,education,221,school
3,20,The English School,0.0,0,39.472603,-0.405620,education,710,school
4,21,Escuela 2,0.0,0,39.478441,-0.401845,education,419,school


In [9]:
vlc_laundry["isochrone_500m"] = vlc_laundry.apply(lambda row: request_isochrone(row["latitude"], row["longitude"], 500, vAPIKEY), axis=1)

In [10]:
def get_habitantes(Code, dataset_ine_valencia):
  is_code = dataset_ine_valencia.loc[:, 'Code'] == Code
  df_code = dataset_ine_valencia.loc[is_code]
  if df_code.empty == False:
    value = df_code['habitantes'].values
  else:
    value = 0.0
  return float(value)

In [11]:
def get_sum_of_poi(row, poi_df,ine_df):
  total_poi = 0.0
  total_habitantes = 0.0
  secciones_censales_lista = []
  for _, r in poi_df.iterrows():
    point1 = Point(r['longitude'], r['latitude'])
    if row['isochrone_500m'].contains(point1) == True:
      row[r['type']] = row[r['type']] + 1.0
      total_poi = total_poi + 1.0
      if r['coddistsec'] not in secciones_censales_lista:
        secciones_censales_lista.append(r['coddistsec'])
        total_habitantes = total_habitantes + get_habitantes(r['coddistsec'],ine_df)

  row['seccensales_poi'] = secciones_censales_lista    
  row['total_poi'] = total_poi
  row['habitantes_total'] = total_habitantes
  return row

In [12]:
uniquevalues = poi_tratados['type'].unique()
vlc_laundry[uniquevalues] = 0.0

In [13]:
vlc_laundry = vlc_laundry.apply(lambda row: get_sum_of_poi(row, poi_tratados,dataset_ine_valencia), axis=1)


In [14]:
vlc_laundry['num_secciones_isocrona'] = vlc_laundry.apply(lambda row: len(row['seccensales_poi']), axis=1)

In [15]:
vlc_laundry

,Unnamed: 0,name,rating,user_ratings_total,latitude,longitude,type,coddistsec,subtypes,isochrone_500m,education,lavanderias,tiendas,transporte,gimnasios,parques,restaurante,bancos,estadios,hospitales,seccensales_poi,total_poi,habitantes_total,num_secciones_isocrona
0,554,Carmen Laundry Quart,4.4,26,39.477416,-0.382865,lavanderias,117,laundry,"POLYGON ((-0.382865 39.48145, -0.383865 39.481...",8.0,1.0,4.0,0.0,1.0,1.0,5.0,0.0,0.0,0.0,"[120, 119, 118, 112, 117, 114, 113, 407, 124, ...",20.0,9135.0,10
1,555,Ruzafa Laundry,4.7,11,39.461641,-0.371781,lavanderias,226,laundry,"POLYGON ((-0.369781 39.46468, -0.371863 39.465...",9.0,4.0,18.0,0.0,3.0,0.0,11.0,4.0,0.0,0.0,"[221, 231, 1013, 201, 225, 215, 208, 206, 226,...",49.0,21216.0,20
2,557,Colada 30,3.9,20,39.463734,-0.398254,lavanderias,822,laundry,"POLYGON ((-0.397254 39.46589, -0.399254 39.465...",4.0,2.0,9.0,0.0,0.0,4.0,3.0,4.0,0.0,0.0,"[732, 725, 828, 822, 837, 813, 836, 823, 826, ...",26.0,13076.0,10
3,558,Bugadry • Valencia,4.5,33,39.462689,-0.389336,lavanderias,840,laundry,"POLYGON ((-0.387336 39.466768, -0.3895 39.4668...",5.0,2.0,9.0,0.0,0.0,1.0,4.0,4.0,0.0,3.0,"[811, 903, 803, 840, 807, 814, 351, 815, 833, ...",28.0,13831.0,13
4,561,Carmen Laundry Serranos,5.0,1,39.479869,-0.377244,lavanderias,111,laundry,"POLYGON ((-0.374244 39.482309, -0.375244 39.48...",3.0,1.0,4.0,1.0,0.0,0.0,4.0,1.0,1.0,0.0,"[103, 510, 114, 111, 517, 118, 110]",15.0,5642.0,7
5,562,Autolavado,0.0,0,39.462768,-0.403386,lavanderias,837,laundry,"POLYGON ((-0.403386 39.467417, -0.404261 39.46...",3.0,1.0,5.0,1.0,0.0,4.0,2.0,4.0,0.0,0.0,"[824, 732, 837, 836, 823, 826, 838, 822]",20.0,10571.0,8
6,563,Lavandería,0.0,0,39.447290,-0.371180,lavanderias,1039,laundry,"POLYGON ((-0.37018 39.450477, -0.373092 39.451...",2.0,2.0,6.0,3.0,0.0,5.0,1.0,0.0,0.0,1.0,"[1043, 1050, 1039, 1027, 1051]",20.0,5799.0,5
7,564,Lavanderías Mr Jeff,4.8,44,39.462717,-0.361575,lavanderias,1001,laundry,"POLYGON ((-0.361575 39.466946, -0.364575 39.46...",5.0,3.0,14.0,0.0,4.0,0.0,6.0,4.0,0.0,0.0,"[257, 1001, 206, 1010, 1012, 1007, 204, 1009, ...",36.0,19430.0,16
8,565,CE Colada Express,5.0,3,39.449530,-0.389459,lavanderias,921,laundry,"POLYGON ((-0.387459 39.454092, -0.390545 39.45...",7.0,2.0,18.0,3.0,2.0,7.0,10.0,3.0,0.0,0.0,"[919, 923, 916, 935, 920, 928, 921, 929, 931, ...",52.0,18768.0,14
9,566,California Laundry,5.0,8,39.461004,-0.371375,lavanderias,226,laundry,"POLYGON ((-0.371375 39.465149, -0.372101 39.46...",9.0,5.0,19.0,0.0,2.0,0.0,11.0,4.0,0.0,0.0,"[221, 232, 231, 1013, 225, 215, 208, 206, 226,...",50.0,19436.0,19


In [20]:
def get_attractivity(row):
  atractivity = (((row['total_poi'])/(row['num_secciones_isocrona']))-((row['lavanderias'])/(row['num_secciones_isocrona']))+((row['habitantes_total'])/(row['num_secciones_isocrona'])))
  return atractivity

In [21]:
vlc_laundry["atractividad"] = vlc_laundry.apply(lambda row: get_attractivity(row), axis=1)

In [22]:
vlc_laundry

,Unnamed: 0,name,rating,user_ratings_total,latitude,longitude,type,coddistsec,subtypes,isochrone_500m,education,lavanderias,tiendas,transporte,gimnasios,parques,restaurante,bancos,estadios,hospitales,seccensales_poi,total_poi,habitantes_total,num_secciones_isocrona,atractividad
0,554,Carmen Laundry Quart,4.4,26,39.477416,-0.382865,lavanderias,117,laundry,"POLYGON ((-0.382865 39.48145, -0.383865 39.481...",8.0,1.0,4.0,0.0,1.0,1.0,5.0,0.0,0.0,0.0,"[120, 119, 118, 112, 117, 114, 113, 407, 124, ...",20.0,9135.0,10,915.400000
1,555,Ruzafa Laundry,4.7,11,39.461641,-0.371781,lavanderias,226,laundry,"POLYGON ((-0.369781 39.46468, -0.371863 39.465...",9.0,4.0,18.0,0.0,3.0,0.0,11.0,4.0,0.0,0.0,"[221, 231, 1013, 201, 225, 215, 208, 206, 226,...",49.0,21216.0,20,1063.050000
2,557,Colada 30,3.9,20,39.463734,-0.398254,lavanderias,822,laundry,"POLYGON ((-0.397254 39.46589, -0.399254 39.465...",4.0,2.0,9.0,0.0,0.0,4.0,3.0,4.0,0.0,0.0,"[732, 725, 828, 822, 837, 813, 836, 823, 826, ...",26.0,13076.0,10,1310.000000
3,558,Bugadry • Valencia,4.5,33,39.462689,-0.389336,lavanderias,840,laundry,"POLYGON ((-0.387336 39.466768, -0.3895 39.4668...",5.0,2.0,9.0,0.0,0.0,1.0,4.0,4.0,0.0,3.0,"[811, 903, 803, 840, 807, 814, 351, 815, 833, ...",28.0,13831.0,13,1065.923077
4,561,Carmen Laundry Serranos,5.0,1,39.479869,-0.377244,lavanderias,111,laundry,"POLYGON ((-0.374244 39.482309, -0.375244 39.48...",3.0,1.0,4.0,1.0,0.0,0.0,4.0,1.0,1.0,0.0,"[103, 510, 114, 111, 517, 118, 110]",15.0,5642.0,7,808.000000
5,562,Autolavado,0.0,0,39.462768,-0.403386,lavanderias,837,laundry,"POLYGON ((-0.403386 39.467417, -0.404261 39.46...",3.0,1.0,5.0,1.0,0.0,4.0,2.0,4.0,0.0,0.0,"[824, 732, 837, 836, 823, 826, 838, 822]",20.0,10571.0,8,1323.750000
6,563,Lavandería,0.0,0,39.447290,-0.371180,lavanderias,1039,laundry,"POLYGON ((-0.37018 39.450477, -0.373092 39.451...",2.0,2.0,6.0,3.0,0.0,5.0,1.0,0.0,0.0,1.0,"[1043, 1050, 1039, 1027, 1051]",20.0,5799.0,5,1163.400000
7,564,Lavanderías Mr Jeff,4.8,44,39.462717,-0.361575,lavanderias,1001,laundry,"POLYGON ((-0.361575 39.466946, -0.364575 39.46...",5.0,3.0,14.0,0.0,4.0,0.0,6.0,4.0,0.0,0.0,"[257, 1001, 206, 1010, 1012, 1007, 204, 1009, ...",36.0,19430.0,16,1216.437500
8,565,CE Colada Express,5.0,3,39.449530,-0.389459,lavanderias,921,laundry,"POLYGON ((-0.387459 39.454092, -0.390545 39.45...",7.0,2.0,18.0,3.0,2.0,7.0,10.0,3.0,0.0,0.0,"[919, 923, 916, 935, 920, 928, 921, 929, 931, ...",52.0,18768.0,14,1344.142857
9,566,California Laundry,5.0,8,39.461004,-0.371375,lavanderias,226,laundry,"POLYGON ((-0.371375 39.465149, -0.372101 39.46...",9.0,5.0,19.0,0.0,2.0,0.0,11.0,4.0,0.0,0.0,"[221, 232, 231, 1013, 225, 215, 208, 206, 226,...",50.0,19436.0,19,1025.315789


In [ ]:
map = folium.Map(location=[39.46994829189022, -0.37787440832473984], zoom_start=12, tiles='CartoDB positron')

In [ ]:
for _, r in vlc_laundry.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['isochrone_500m']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['isochrone_500m']).add_to(geo_j)
    geo_j.add_to(map)

In [ ]:
map